## Q1

In [ ]:
######### my version (passed) #########

from abc import ABC
from collections import deque
import random
random.seed(123)

class TYPE:
    MarketData = 0
    MarketResponse = 1
    
class ApplicationBase(ABC):
    def __init__(self):
        self.channels = {}
    def add_channel(self,name,channel):
        self.channels[name] = channel

# For the library random, the seed is set to 123. Please do not change this value.

# Do not forget to create the two exceptions

class ChannelEmptyError(Exception):
    pass

class ChannelFullError(Exception):
    pass

class Channel:
    # define the constructor
    # code the function read
    def __init__(self,size_limit = 10):
    
        self.chan = deque()
        self.size_limit = size_limit
        
    def read(self):
        
        if len(self.chan)==0:
            raise ChannelEmptyError("Channel Empty")
        else:
            return self.chan.popleft()

    # code the function write
    def write(self,item):
        
        if len(self.chan)+1<=self.size_limit:
            self.chan.append(item)
        else:
            raise ChannelFullError("Channel Full")
    

class MarketSimulator(ApplicationBase):
    # code the constructor
    def __init__(self):
        super().__init__()
        
    # code the following functions
    def subscribe(self,name,channel):
        self.add_channel(name,channel)
        
    def send_market_data(self,market_data):
        for c_name in list(self.channels.keys()):
            c = self.channels[c_name]
            c.write(market_data)
        
    def generate_random_market_data(self):
        
        random_market_data = {}
        random_market_data['type'] = 0
        random_market_data['price'] = random.randint(2,9)
        self.send_market_data(random_market_data)
        
    def send_market_response(self,channel_name,market_response):
        
        c = self.channels[channel_name]
        c.write(market_response)
    

class MATradingStrategy(ApplicationBase):
    # code the constructor 
    def __init__(self,window_time):
        
        super().__init__()
        self.window_time = window_time
        self.market_data = []
        
    # code the moving average function. do not forget to round to 2 numbers
    def calculate_mavg(self,number):
        
        number = number[(-1)*self.window_time:]
        mavg = round(sum(number)/len(number),2)
        
        return mavg
        
    # code the following functions
    def subscribe(self,name,channel):
        self.add_channel(name,channel)
        
    def read_channel(self):
        
        for c_name in list(self.channels.keys()):
            c = self.channels[c_name]
            info = c.read()
            
            # print(info)
            
            if len(info)==2:
                self.onMarketDataReceived(info)
            else:
                self.onMarketResponse(info)
        
    def onMarketDataReceived(self, marketdata):
        self.market_data.append(marketdata)
        
        market_data_list = []
        for m in self.market_data:
            market_data_list.append(m['price'])
        
        print(self.calculate_mavg(market_data_list))
    
    def onMarketResponse(self, marketresponse):
        
        # md1={'type' : TYPE.MarketResponse, 'state' : 'Open', 'id' : 1}
        
        result = "Order: "+str(marketresponse["id"])+" State: "+str(marketresponse["state"])
        # response = {}
        # response['Order'] = marketresponse["id"]
        # response['State'] = marketresponse["state"]
        
        print(result)


        
def test_channel_communication_1():
    c1=Channel()
    c1.write(1)
    c1.write(2)
    c1.write(3)
    print(c1.read())
    print(c1.read())
    print(c1.read())

def test_channel_communication_2():
    try:
        c1=Channel(size_limit=2)
    except Exception:
        "It is possible you forgot an arugment for the constructor"
    c1.write(1)
    c1.write(2)
    try:
        c1.write(3)
    except ChannelFullError as e:
        print(e)
    print(c1.read())
    print(c1.read())


def test_channel_communication_3():
    c1=Channel()
    for i in range(10):
        c1.write(i)
    try:
        c1.write(11)
    except ChannelFullError as e:
        print(e)
    for i in range(10):
        print(c1.read())


def test_channel_communication_4():
    c1=Channel()
    for i in range(10):
        c1.write(i)
    for i in range(10):
        print(c1.read())
    try:
        c1.read()
    except ChannelEmptyError as e:
        print(e)


def test_market_data_generator_1():
    md1=MarketSimulator()
    md_2_ts=Channel()
    md1.subscribe("trading_strategy1",md_2_ts)
    md1.generate_random_market_data()
    print(md_2_ts.read())

def test_market_data_generator_2():
    md1=MarketSimulator()
    md_2_ts=Channel()
    md1.subscribe("trading_strategy1",md_2_ts)
    for _ in range(10):
        md1.generate_random_market_data()
    for _ in range(10):
        print(md_2_ts.read())

def test_market_data_generator_3():
    md1=MarketSimulator()
    md_2_ts1=Channel()
    md_2_ts2=Channel()
    md1.subscribe("trading_strategy1",md_2_ts1)
    md1.subscribe("trading_strategy2",md_2_ts2)

    for _ in range(10):
        md1.generate_random_market_data()
    for _ in range(10):
        print(md_2_ts1.read())
        print(md_2_ts2.read())




def test_strategy_1():
    ma1=MATradingStrategy(2)
    md1={'type' : TYPE.MarketData, 'price' : 1}
    ma1.onMarketDataReceived(md1)
    md1 = {'type': TYPE.MarketData, 'price': 3}
    ma1.onMarketDataReceived(md1)
    md1 = {'type': TYPE.MarketData, 'price': 5}
    ma1.onMarketDataReceived(md1)

def test_strategy_2():
    ma1=MATradingStrategy(3)
    md1={'type' : TYPE.MarketData, 'price' : 1}
    ma1.onMarketDataReceived(md1)
    md1 = {'type': TYPE.MarketData, 'price': 3}
    ma1.onMarketDataReceived(md1)
    md1 = {'type': TYPE.MarketData, 'price': 5}
    ma1.onMarketDataReceived(md1)
    md1 = {'type': TYPE.MarketData, 'price': 7}
    ma1.onMarketDataReceived(md1)
    md1 = {'type': TYPE.MarketData, 'price': 9}
    ma1.onMarketDataReceived(md1)


def test_strategy_3():
    ma1=MATradingStrategy(2)
    md1={'type' : TYPE.MarketResponse, 'state' : 'Open', 'id' : 1}
    ma1.onMarketResponse(md1)

def test_strategy_4():
    md1=MarketSimulator()
    md_2_ts=Channel()
    md1.subscribe("trading_strategy1",md_2_ts)
    mr1 = {'type': TYPE.MarketResponse, 'state': 'Open', 'id': 1}
    md1.send_market_response("trading_strategy1",mr1)
    print(md_2_ts.read())


def test_strategy_5():
    md1=MarketSimulator()
    ts1=MATradingStrategy(2)
    md_2_ts=Channel()
    md1.subscribe("trading_strategy1",md_2_ts)
    ts1.subscribe("trading_strategy1",md_2_ts)
    md1.generate_random_market_data()
    ts1.read_channel()
    md1.generate_random_market_data()
    ts1.read_channel()
    md1.generate_random_market_data()
    ts1.read_channel()
    mr1 = {'type': TYPE.MarketResponse, 'state': 'Open', 'id': 1}
    md1.send_market_response("trading_strategy1",mr1)
    ts1.read_channel()

def test_strategy_6():
    md1=MarketSimulator()
    ts1=MATradingStrategy(2)
    md_2_ts=Channel()
    md1.subscribe("trading_strategy1",md_2_ts)
    ts1.subscribe("trading_strategy1",md_2_ts)
    for i in range(10):
        md1.generate_random_market_data()
        ts1.read_channel()
        md1.generate_random_market_data()
        ts1.read_channel()
        if i%3==0:
            mr1 = {'type': TYPE.MarketResponse, 'state': 'Open', 'id': i}
            md1.send_market_response("trading_strategy1",mr1)
            ts1.read_channel()


def test_strategy_7():
    md1=MarketSimulator()
    ts1=MATradingStrategy(2)
    ts2 = MATradingStrategy(4)

    md_2_ts=Channel()
    md1.subscribe("trading_strategy1",md_2_ts)
    ts1.subscribe("trading_strategy1",md_2_ts)

    md_2_ts2 = Channel()
    md1.subscribe("trading_strategy2", md_2_ts2)
    ts2.subscribe("trading_strategy2", md_2_ts2)

    for i in range(10):
        md1.generate_random_market_data()
        ts1.read_channel()
        ts2.read_channel()
        md1.generate_random_market_data()
        ts1.read_channel()
        ts2.read_channel()
        if i%3==0:
            mr1 = {'type': TYPE.MarketResponse, 'state': 'Open', 'id': i}
            md1.send_market_response("trading_strategy1",mr1)
            ts1.read_channel()
            try:
                ts2.read_channel()
            except ChannelEmptyError as e:
                pass

            
def check_inheritance():
    assert(issubclass(MarketSimulator, ApplicationBase))
    assert(issubclass(MATradingStrategy, ApplicationBase))
            
if __name__ == '__main__':
    func_name = input().strip()
    globals()[func_name]()

In [ ]:
######### Solution #########

#Q1

# For the library random, the seed is set to 123. Please do not change this value.
# Do not forget to create the two exceptions
class ChannelEmptyError(Exception):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
class ChannelFullError(Exception):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

class Channel:
    def __init__(self, size_limit = 10):
        self.data = deque()
        self.sizel = size_limit
    def read(self):
        if len(self.data) == 0:
            raise ChannelEmptyError("Channel Empty")
        else:
            return self.data.pop()
    def write(self,item):
        if len(self.data) == self.sizel:
            raise ChannelFullError("Channel Full")
        else:
            self.data.appendleft(item)
    

class MarketSimulator(ApplicationBase):
    def __init__(self):
        super().__init__()
    def subscribe(self,name,channel):
        self.add_channel(name, channel)
    def send_market_data(self,market_data):
        for keys in self.channels.keys():
            self.channels[keys].write(market_data)    
    def generate_random_market_data(self):
        price = random.randint(2,9)
        self.send_market_data({"type":TYPE.MarketData,"price":price})   
    def send_market_response(self,channel_name,market_response):
        self.channels[channel_name].write(market_response)
        
    
class MATradingStrategy(ApplicationBase):
    def __init__(self,window_time):
        super().__init__()
        self.window_time = window_time
        self.price = []
    def calculate_mavg(self,number):
        self.price.append(number)
        start = max(0, len(self.price)-self.window_time)
        mavg = 0
        for a in range(start, len(self.price)):
            mavg += self.price[a]
        if start == 0:
            print(round(mavg/len(self.price),2))
        else:
            print(round(mavg/self.window_time,2))
    def subscribe(self,name,channel):
        self.add_channel(name, channel)   
    def read_channel(self):
        for i, channel in self.channels.items():
            data = channel.read()
            if data["type"] == TYPE.MarketData:
                self.onMarketDataReceived(data)
            else: 
                self.onMarketResponse(data)        
    def onMarketDataReceived(self, marketdata):
        self.calculate_mavg(marketdata["price"])
    def onMarketResponse(self, marketresponse):
        print(f'Order: {marketresponse["id"]} State: {marketresponse["state"]}')

## Q2

In [ ]:
######### my version (passed) #########


#!/bin/python3

import math
import os
import random
import re
import sys


#
# Complete the 'getMaxDeletions' function below.
#
# The function is expected to return an INTEGER.
# The function accepts STRING s as parameter.
#

def getMaxDeletions(s):
    # Write your code here
    n = len(s)
    left,right,up,down = 0,0,0,0

    for i in range(n):
        if s[i] == 'L':
            left += 1
        elif s[i] == 'R':
            right += 1 
        elif s[i] == 'U':
            up += 1
        else:
            down += 1
    
    result = 2*(min(left,right) + min(up,down))

    return result
if __name__ == '__main__':
    fptr = open(os.environ['OUTPUT_PATH'], 'w')

    s = input()

    result = getMaxDeletions(s)

    fptr.write(str(result) + '\n')

    fptr.close()


In [ ]:
######### Solution #########

#Q2
#
# Complete the 'getMaxDeletions' function below.
#
# The function is expected to return an INTEGER.
# The function accepts STRING s as parameter.
#
from collections import Counter
def getMaxDeletions(s):
    # Write your code here
    steps = Counter(s)
    return len(s) - (abs(steps['U'] - steps['D']) + abs(steps['L'] - steps['R']))